# AtomsBase integration

[AtomsBase.jl](https://github.com/JuliaMolSim/AtomsBase.jl) is a common interface
for representing atomic structures in Julia. DFTK directly supports using such
structures to run a calculation as is demonstrated here.

In [1]:
using DFTK

## Feeding an AtomsBase AbstractSystem to DFTK
In this example we construct a silicon system using the `ase.build.bulk` routine
from the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
(ASE), which is exposed by [ASEconvert](https://github.com/mfherbst/ASEconvert.jl)
as an AtomsBase `AbstractSystem`.

In [2]:
# Construct bulk system and convert to an AbstractSystem
using ASEconvert
system_ase = ase.build.bulk("Si")
system = pyconvert(AbstractSystem, system_ase)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0    2.715    2.715;
                            2.715        0    2.715;
                            2.715    2.715        0]u"Å"

    Atom(Si, [       0,        0,        0]u"Å")
    Atom(Si, [  1.3575,   1.3575,   1.3575]u"Å")


To use an AbstractSystem in DFTK, we attach pseudopotentials, construct a DFT model,
discretise and solve:

In [3]:
system = attach_psp(system; Si="hgh/lda/si-q4")

model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921663237104                   -0.69    6.0         
  2   -7.926158425029       -2.35       -1.22    1.0    407ms
  3   -7.926836445658       -3.17       -2.37    1.9    382ms
  4   -7.926861479698       -4.60       -3.03    2.6    397ms
  5   -7.926861631998       -6.82       -3.36    1.8    335ms
  6   -7.926861664852       -7.48       -3.69    2.0    360ms
  7   -7.926861680341       -7.81       -4.34    1.0    344ms
  8   -7.926861681731       -8.86       -4.75    2.1    359ms
  9   -7.926861681857       -9.90       -5.22    1.4    326ms
 10   -7.926861681871      -10.86       -5.86    2.0    354ms
 11   -7.926861681872      -11.93       -6.27    2.1    396ms
 12   -7.926861681873      -12.76       -6.88    2.0    352ms
 13   -7.926861681873      -14.27       -7.79    1.9    357ms
 14   -7.926861681873      -15.05       -7.95    3.2    421ms
 15   -7.

If we did not want to use ASE we could of course use any other package
which yields an AbstractSystem object. This includes:

### Reading a system using AtomsIO

In [4]:
using AtomsIO

# Read a file using [AtomsIO](https://github.com/mfherbst/AtomsIO.jl),
# which directly yields an AbstractSystem.
system = load_system("Si.extxyz")

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921686510836                   -0.69    5.8         
  2   -7.926162555042       -2.35       -1.22    1.0    339ms
  3   -7.926836412139       -3.17       -2.37    1.8    367ms
  4   -7.926861521185       -4.60       -3.01    3.0    509ms
  5   -7.926861630172       -6.96       -3.34    1.5    335ms
  6   -7.926861665950       -7.45       -3.71    1.9    352ms
  7   -7.926861680695       -7.83       -4.42    1.4    332ms
  8   -7.926861681820       -8.95       -5.02    2.4    427ms
  9   -7.926861681867      -10.33       -5.38    1.9    362ms
 10   -7.926861681872      -11.33       -5.91    1.8    337ms
 11   -7.926861681873      -12.06       -7.01    1.9    380ms
 12   -7.926861681873      -13.48       -7.78    3.1    465ms
 13   -7.926861681873   +    -Inf       -8.55    2.5    388ms


The same could be achieved using [ExtXYZ](https://github.com/libAtoms/ExtXYZ.jl)
by `system = Atoms(read_frame("Si.extxyz"))`,
since the `ExtXYZ.Atoms` object is directly AtomsBase-compatible.

### Directly setting up a system in AtomsBase

In [5]:
using AtomsBase
using Unitful
using UnitfulAtomic

# Construct a system in the AtomsBase world
a = 10.26u"bohr"  # Silicon lattice constant
lattice = a / 2 * [[0, 1, 1.],  # Lattice as vector of vectors
                   [1, 0, 1.],
                   [1, 1, 0.]]
atoms  = [:Si => ones(3)/8, :Si => -ones(3)/8]
system = periodic_system(atoms, lattice; fractional=true)

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921658481148                   -0.69    5.6         
  2   -7.926174470002       -2.35       -1.22    1.0    314ms
  3   -7.926840576949       -3.18       -2.37    1.8    360ms
  4   -7.926864913413       -4.61       -3.03    2.6    400ms
  5   -7.926865051202       -6.86       -3.38    2.0    396ms
  6   -7.926865079611       -7.55       -3.76    1.9    336ms
  7   -7.926865091425       -7.93       -4.31    1.2    302ms


## Obtaining an AbstractSystem from DFTK data

At any point we can also get back the DFTK model as an
AtomsBase-compatible `AbstractSystem`:

In [6]:
second_system = atomic_system(model)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0     5.13     5.13;
                             5.13        0     5.13;
                             5.13     5.13        0]u"a₀"

    Atom(Si, [  1.2825,   1.2825,   1.2825]u"a₀")
    Atom(Si, [ -1.2825,  -1.2825,  -1.2825]u"a₀")


Similarly DFTK offers a method to the `atomic_system` and `periodic_system` functions
(from AtomsBase), which enable a seamless conversion of the usual data structures for
setting up DFTK calculations into an `AbstractSystem`:

In [7]:
lattice = 5.431u"Å" / 2 * [[0 1 1.];
                           [1 0 1.];
                           [1 1 0.]];
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

third_system = atomic_system(lattice, atoms, positions)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0  5.13155  5.13155;
                          5.13155        0  5.13155;
                          5.13155  5.13155        0]u"a₀"

    Atom(Si, [ 1.28289,  1.28289,  1.28289]u"a₀")
    Atom(Si, [-1.28289, -1.28289, -1.28289]u"a₀")
